In [4]:
pip install oauth2client

In [5]:
pip install gspread

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pyttsx3

Note: you may need to restart the kernel to use updated packages.


In [8]:
# Importing required Libraries
import requests
import pdfplumber
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from googletrans import Translator
import datetime
import pyttsx3
import tkinter as tk
from tkinter import filedialog
from glob import glob
import os

In [3]:
#Open multiple files and set engine (text to speech) properties
multiple_files = glob ("C:/Users/63927/Desktop/New folder/*pdf")
engine = pyttsx3.init ()
engine.setProperty ('rate', 160)
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

translater = Translator ()

In [4]:
#Set properties for google sheet API
scope = ['https://www.googleapis.com/auth/spreadsheets',
        "https://www.googleapis.com/auth/drive.file",
        "https://www.googleapis.com/auth/drive"]

cred = ServiceAccountCredentials.from_json_keyfile_name('Credentials.json', scope)
client = gspread.authorize(cred)
python_write = client.open("Receiving Waybill").worksheet("April 2022")

In [19]:
# Open each invoice in pdf format and print
for i in multiple_files:
    with pdfplumber.open (i)as pdf:
        page = pdf.pages [0]
        waybill = page.extract_text()
        #print (waybill)
        print(os.path.splitext("C:/Users/63927/Desktop/New folder/")[0])
        
# Iterate for each invoice and do the following data cleaning and sorting      
        
        for row in waybill.split ("\n"):
            if row.startswith ("箱上编号"):
                stra_code = row.split ()[1]

        for row in waybill.split ("\n"):
            if row.endswith ("WAYBILL DETAILS"):
                waybill_number = row.split ()[0]

        for row in waybill.split ("\n"):
            if row.startswith ("收货日期:"):
                date= row.split ()[1]


        product_names = []
        cargo_specifications = []
        total_packages = []
        boxes_bags = []
        total_weights = []
        cubic_numbers = []
        remarks = []
        values = ("1 ","2 ","3 ", "4 ", "5 ", "6 ","7 ","8 ","9 ","10 ")

        for row in waybill.split ("\n"):       
            if row.startswith (values):
                name = row.split ()[1]
                specification = row.split ()[2]
                packages = row.split ()[3]
                box_bag = row.split ()[4]
                total_weight = row.split ()[5]
                cubic_number = row.split ()[6]
                remark = row.split ()[-1]

                product_names.append (name)
                cargo_specifications.append (specification)
                total_packages.append (packages)
                boxes_bags.append (box_bag)
                total_weights.append (total_weight)
                cubic_numbers.append (cubic_number)
                remarks.append (remark) 
        print ("Stra code:",stra_code)
        print ("Waybill # :",waybill_number)
        print ("Waybill date:",date)
        print ("Product Name:", product_names)
        print ("Cargo Specification:", cargo_specifications)
        print ("Total Cases/Packages:", total_packages)
        print ("Box/Bag KG:", boxes_bags)
        print ("Total Weight:", total_weights)
        print ("Cubic Number:", cubic_numbers)
        print ("*******************************************************")  
        
        # Translate product names from chineese to english
        product_names_translated = []
        for i in product_names:
            new = translater.translate((i), dest = "en")
            product_names_translated.append (new.text)

        product_names_translated

        # Do final operations and append all informations from this console to google sheets
        stra_codes = len(product_names)*(stra_code,)
        waybill_numbers=len(product_names)*(waybill_number,)
        dates = len(product_names)*(date,)
        warehouse_number= waybill_number [0]

        if warehouse_number== "1":
            warehouse_text = "Shishi"
        elif warehouse_number == "2":
            warehouse_text = "Guangzhou"
        else:
            warehouse_text = "Yiwu"

        warehouse_texts = len (product_names)*(warehouse_text,)

        current_date = datetime.datetime.now()
        current_date_str = current_date.strftime("%m/%d/%Y  %H:%M:%S")
        current_date_strs = len (product_names)*(current_date_str,)

        #engine.say ("Appending Waybill {}.". format (i))
        #engine.runAndWait ()
        
        for i in range (0,len(product_names)):
            lst = [current_date_strs,stra_codes, waybill_numbers,warehouse_texts,dates,product_names,product_names_translated ,cargo_specifications, total_packages, boxes_bags, total_weights,cubic_numbers]
            lst2 = []
            lst2.append([x[i] for x in lst])
            print (lst2[0])
            python_write.append_row (lst2[0])

        # Checking for a potential error upon running the progrma
        if i == len (product_names)-1:
            engine.say ("waybill successfully appended. Thank you!")
            engine.runAndWait ()
        else:
            engine.say ("There's an error upon running this program. Please check. Thank you!")
            engine.runAndWait ()

            print ("*******************************************************")  

C:/Users/63927/Desktop/New folder/
Stra code: STRA-4318
Waybill # : 3204130030
Waybill date: 2022-04-13
Product Name: ['脚轮-钢', '脚轮', '脚轮', '脚轮', '脚轮']
Cargo Specification: ['34.00*50.00*10.00', '36.00*44.00*13.00', '60.00*52.00*23.00', '50.00*55.00*16.00', '32.00*45.00*16.00']
Total Cases/Packages: ['1', '2', '1', '1', '1']
Box/Bag KG: ['28.00', '28.00', '28.00', '26.00', '28.00']
Total Weight: ['28.00', '56.00', '28.00', '26.00', '28.00']
Cubic Number: ['0.02', '0.05', '0.08', '0.05', '0.03']
*******************************************************
['04/27/2022  11:42:36', 'STRA-4318', '3204130030', 'Yiwu', '2022-04-13', '脚轮-钢', 'Casters - Steel', '34.00*50.00*10.00', '1', '28.00', '28.00', '0.02']
['04/27/2022  11:42:36', 'STRA-4318', '3204130030', 'Yiwu', '2022-04-13', '脚轮', 'casters', '36.00*44.00*13.00', '2', '28.00', '56.00', '0.05']
['04/27/2022  11:42:36', 'STRA-4318', '3204130030', 'Yiwu', '2022-04-13', '脚轮', 'casters', '60.00*52.00*23.00', '1', '28.00', '28.00', '0.08']
['04/2

In [20]:
type (multiple_files)

list